In [2]:
from rdflib import Graph, Namespace, URIRef

In [3]:
filename = 'm2j_output/objects/Brick/1.3/BESTESTAir/TestCases/TestCase_Ideal.ttl'

In [4]:
g = Graph()
g.parse(filename, format = 'ttl')
BLDG = Namespace("https://BESTESTAir.urn#")
BR = Namespace("https://boptest-rules.urn#")
BRICK = Namespace("https://brickschema.org/schema/Brick#")
g.bind('bldg',BLDG)

In [5]:
pointOfquery = """
PREFIX brick: <https://brickschema.org/schema/Brick#> 
PREFIX boptestrules: <https://boptest-rules.urn#>
CONSTRUCT {
    ?part brick:hasPoint ?point .
}
WHERE {
   ?point boptestrules:pointOf ?type .
    ?parent brick:hasPart ?part .
   ?part a ?type .
}
"""

In [6]:
construct = g.query(pointOfquery)
g + construct.graph

<Graph identifier=N5599f6fb640b4265bebe00b55c84a1de (<class 'rdflib.graph.Graph'>)>

In [7]:
print(construct.graph.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<https://BESTESTAir.urn#fcu_fan> brick:hasPoint <https://BESTESTAir.urn#fcu_oveFan>,
        <https://BESTESTAir.urn#fcu_reaPFan> .




In [8]:
# getting rid of all references to the sameAs point, or the weird points it creates.
for keeper, sameAs_pred, remover in g.triples((None, BR['sameAs'], None)):
    print('Keeper: ',keeper,' | Remover: ', remover)
    for remover, hasPoint_pred, fake_points in g.triples((URIRef(remover), BRICK['hasPoint'], None)):
        print('fake points: ', fake_points)
        g.remove((remover, hasPoint_pred, fake_points))
    g.remove((keeper, sameAs_pred, remover))
    

Keeper:  https://BESTESTAir.urn#con  | Remover:  https://BESTESTAir.urn#con_Thermostat_T
fake points:  https://BESTESTAir.urn#con_Thermostat_T_oveTSetCoo
fake points:  https://BESTESTAir.urn#con_Thermostat_T_oveTSetHea
Keeper:  https://BESTESTAir.urn#fcu  | Remover:  https://BESTESTAir.urn#fcu_FanCoilUnit_T
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_oveFan
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_oveTSup
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_reaFloSup
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_reaPCoo
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_reaPFan
fake points:  https://BESTESTAir.urn#fcu_FanCoilUnit_T_reaPHea


In [9]:
print(g.serialize())

@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix literal: <https://literal_values.urn#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg:con a brick:Thermostat ;
    brick:hasPoint bldg:con_oveTSetCoo,
        bldg:con_oveTSetHea .

bldg:fcu a brick:FCU ;
    brick:hasPart bldg:fcu_fan ;
    brick:hasPoint bldg:fcu_oveFan,
        bldg:fcu_oveTSup,
        bldg:fcu_reaFloSup,
        bldg:fcu_reaPCoo,
        bldg:fcu_reaPFan,
        bldg:fcu_reaPHea .

bldg:fcu_FanCoilUnit_T brick:hasPart bldg:fcu_FanCoilUnit_T_fan .

bldg:con_oveTSetCoo a brick:Zone_Air_Cooling_Temperature_Setpoint ;
    qudt:hasQuantityKind quantitykind:Temperature ;
    qudt:hasUnit qudt:K ;
    ref:hasExternalReference bldg:con_o

In [10]:
g.serialize('post-processed-model.ttl', format = 'ttl')

<Graph identifier=N2b972dba9e764c3da86589c23d4db765 (<class 'rdflib.graph.Graph'>)>